In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from utils import DEint as de

In [3]:
raw_eqs = pd.read_csv('RMHD_CHM_DE.csv', header=None)
pd.set_option('display.max_colwidth', None)
raw_eqs.columns = ['eqn']

In [4]:
type(raw_eqs)

pandas.core.frame.DataFrame

In [5]:
# Strip repetetive ' == 0' from the equations

raw_eqs.eqn = raw_eqs.eqn.apply(lambda eq: eq.strip(' == 0'))
constants = de.find_constants(raw_eqs.eqn)
print(constants)
raw_eqs.eqn.iloc[3850]
# term = de.process_term('Derivative[0, 2, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8]', constants)
# term[0]
# type(term[0]) != int
# de.term_to_dict(term)

['α', 'η']


'2*Derivative[0, 1, 0, 0, 0, 0, 1, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*α*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - Derivative[0, 2, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[1, 2, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8] - 3*Derivative[2, 0, 0, 0, 0, 0, 0, 1][xse2][z1, z2, z3, z4, z5, z6, z7, z8] - α*Derivative[3, 0, 0, 0, 0, 0, 0, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8]'

In [6]:
# Separating the terms and signs 

raw_eqs['Signs'] = raw_eqs.eqn.apply(de.get_signs)
raw_eqs['Terms'] = raw_eqs.eqn.apply(de.get_terms)

raw_eqs['Signs']

0                              [-1]
1                               [1]
2                              [-1]
3                               [1]
4                              [-1]
                   ...             
3850        [1, -1, -1, -1, -1, -1]
3851        [1, -1, -1, -1, -1, -1]
3852    [-1, -1, 1, 1, -1, 1, 1, 1]
3853        [-1, -1, 1, -1, -1, -1]
3854    [-1, -1, -1, 1, -1, -1, -1]
Name: Signs, Length: 3855, dtype: object

In [7]:
term = de.process_term('α^2*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8]', constants)
de.term_to_dict(term)

{'coefficient': 1,
 'constants': [2, 0],
 'derivatives': [0, 1, 1, 0, 0, 0, 1, 0],
 'variable': 'xse5'}

In [8]:
de.intlist('[0, 1, 1, 0, 0, 0, 1, 0]')

[0, 1, 1, 0, 0, 0, 1, 0]

In [9]:
A = "Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5]".strip("Derivative")
A

'[0, 1, 1, 0, 0, 0, 1, 0][xse5]'

In [10]:
de.process_derivative("Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5]")

('xse5', [0, 1, 1, 0, 0, 0, 1, 0])

In [11]:
eqn = "Derivative[0, 1, 0, 0, 0, 0, 1, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8] - 2*α*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8]"

In [12]:
de.get_terms(eqn)

['Derivative[0, 1, 0, 0, 0, 0, 1, 1][eta2][z1, z2, z3, z4, z5, z6, z7, z8]',
 '2*α*Derivative[0, 1, 1, 0, 0, 0, 1, 0][xse5][z1, z2, z3, z4, z5, z6, z7, z8]']

In [13]:
de.get_signs(eqn)

[1, -1]

In [14]:
de.eqn_process(eqn, constants)

[{'coefficient': 1,
  'constants': [0, 0],
  'derivatives': [0, 1, 0, 0, 0, 0, 1, 1],
  'variable': 'eta2'},
 {'coefficient': -2,
  'constants': [1, 0],
  'derivatives': [0, 1, 1, 0, 0, 0, 1, 0],
  'variable': 'xse5'}]

In [31]:
det_eqn = []
for i, row in raw_eqs.iterrows():
    det_eqn.append(de.eqn_process(row.eqn, constants))
det_eqn[140]

[{'coefficient': -4,
  'constants': [0, 0],
  'derivatives': [0, 0, 0, 0, 0, 0, 1, 1],
  'variable': 'xse1'}]

In [16]:
A = [0,0,0,1,0,0]
B = [3,0,1,1,0,0]
de.compare_derivatives(A,B)

True

In [25]:
A = {1 :"a", 2: "b", 3:"c", 4: []}
for i, v in A.items():
    print(i,v)

1 a
2 b
3 c
4 []


In [18]:
len([])

0

In [19]:
A.pop(1)

'a'

In [20]:
A = [0,0,0]

In [21]:
A [A.index("a")] = 0

ValueError: 'a' is not in list

In [22]:
A

[0, 0, 0]

In [23]:
list(filter(lambda num: num != 0, A))

[]

In [26]:
new_dict = {k:v for k,v in A.items() if v}

In [27]:
len(new_dict)

3

In [32]:
import Lie_Analysis as LA

In [33]:
det_eqns = LA.cvs_to_list()

In [35]:
det_eqns[140][0]

{'coefficient': -4,
 'constants': [0, 0],
 'derivatives': [0, 0, 0, 0, 0, 0, 1, 1],
 'variable': 'xse1'}